### 5.6.1 面向对象游戏引擎

这个“游戏引擎”类中还包含窗口、精灵等对象，每个对象也都具有自己的数据属性和功能属性，比如窗口有长宽、标题、背景和前景颜色，可以在窗口的画布上绘制像素等，精灵也有自己的位置、图像、速度、运动等属性。“游戏引擎”和窗口、精灵之间是一种包含关系。
1.	游戏引擎类

```
其主要包含的属性有：
数据属性：
 游戏窗口（屏幕）window：窗口长宽、标题、绘制表面、背景或前景图像或颜色、字体颜色等
 所有的精灵Spirit 
 
功能属性：
初始化init()
    游戏主循环run()
         事件处理processEvent() 
         更新数据并处理碰撞collision()
         绘制场景render()
退出游戏quit()
```


In [3]:
import random
import pygame, sys
from pygame.locals import *

#----游戏状态常量--------
RUNNING = 1   #是否正在运行
PAUSE = -1    #暂停
STOP = 0      #停止

FPS  = 25  #画面刷新速率  

#----颜色 常量--------
BLACK = (0,0,0)
WHITE = (255, 255, 255)
GREEN = (0, 60, 0)
GREY = (210, 210 ,210)
RED = (255, 0, 0)
PURPLE = (255, 0, 255)
YELLOW = (120,120,0)

class GameEngine():
    #=========1. 初始化=============   
    def __init__(self, width, height,title = 'game Engine',background = None,foreground = None): 
        self.width, self.height,self.title = width, height,title
        pygame.init()       
        pygame.mixer.init()    #初始化播放声音引擎
        self.surface = pygame.display.set_mode((width, height))  
        pygame.display.set_caption(title)   #设置窗口标题
        self.runningState = RUNNING
        
        #------初始化背景或前景 background and foreground----
     #   self.background = background
     #   self.foreground = foreground
        self.set_background(background)                          
        self.set_foreground(foreground)  
        self.bg_color = BLACK    #背景颜色
        self.fg_color = YELLOW   #前景颜色
        self.logo_image = None
        
        #-------初始化游戏数据---------
        self.sprites_list = []       #初始化精灵数组 
        #...

#=========2. 游戏主循环===========
    def run(self):
        self.game_intro()
        clock = pygame.time.Clock()     #时钟        
        self.running = RUNNING
        while self.running != STOP:           
            self.running = self.processEvent()   # 2.1 事件处理
            if self.running == RUNNING:
                self.update()                # 2.2.1 更新数据
                self.collosion()              # 2.2.2 碰撞检测
                self.draw()                  # 2.3绘制场景
                pygame.display.flip()         # pygame.display.update()  
        
        clock.tick(FPS)                #每秒的帧频率FPS Frames
        pygame.quit()                   # 3. 退出程序
        
    def game_intro(self):
        pass      
                
    def set_foreground(self, image=None):
        self.foreground = image
        if image: 
            self.foreground = pygame.image.load(image)
         
    def set_background(self, image=None):
        self.background = image
        if image: 
            self.background = pygame.image.load(image)
    
    def init(self):        
        pass
    def add_sprite(self,sprite):
        self.sprites_list.append(sprite)
        
    def remove_sprite(self,sprite):
        self.sprites_list.remove(sprite)   
            
    #=======2.1 处理（键盘、鼠标等）事件===========    
    def processEvent(self):
        for event in pygame.event.get():     #返回当前的所有事件 
            if event.type == pygame.QUIT:  #接收到窗口关闭事件
                return STOP                #退出游戏 
            elif event.type == KEYDOWN:              
                return self.keydown(event)
            elif event.type == KEYUP:
                return self.keyup(event)
        return RUNNING                         #正常  
    
    #  键按下处理函数 keydown handler
    def keydown(self,event):
        if event.key == pygame.K_ESCAPE:
            return STOP     #退出
        return 1    #正常

    # 键弹起处理函数 keyup handler
    def keyup(self,event):
        return 1
        
    #===========2.2.1 更新数据================
    def update(self):
        for sprite in self.sprites_list:
            sprite.update()     
        for sprite in self.sprites_list:
            if sprite.is_dead():
                self.remove_sprite(sprite)    
    
    #===========2.2.2 碰撞检测处理================
    def collosion(self):
        pass
   
    #===========2.3 绘制场景=========
    def draw(self):        
        self.draw_background(self.surface)
        self.draw_foreground(self.surface)
        self.draw_sprites(self.surface) 
        self.draw_scores(self.surface) 
        #self.player.draw(self.surface)
 
    def draw_sprites(self, surface):       
        for sprite in self.sprites_list:
            sprite.draw(surface)       

    def draw_background(self, surface):
        surface.fill(self.bg_color)   # clear surface to green
        if self.background:
            surface.blit(self.background, (0,0))
            
    def draw_foreground(self, surface):
        if self.foreground:
            surface.blit(self.foreground, (0,0))   
 
    def draw_scores(self, surface):
        pass

    #在位置pos处用字体fontname和fontsize绘制文本txt
    def draw_text(self,text,pos,fontname = "Comic Sans MS", fontsize=20):
        #绘制得分 scores
        myfont1 = pygame.font.SysFont(fontname, fontsize)
        label1 = myfont1.render(text, 1, (255,255,0))
        self.surface.blit(label1, pos)  

### 2.	精灵

精灵是游戏中的主要角色，有各种各样的精灵（挡板、球、飞机、炮弹等），可以定义一个最基本的精灵类，表示所有不同种类精灵的共同特性，在此基础上再派生出特殊的精灵类。
精灵的共同属性有：（图片表示）的形象、位置、绘制（在屏幕上显示自己）、更新状态。

In [4]:
#=============所有精灵类的基类=================
class Sprite():
    def __init__(self, image=None, pos=(0,0),vel = (0,0)):
        self.pos = [pos[0],pos[1]]
        self.vel = [vel[0],vel[1]]       
        self.lives = 1
        self.set_image(image)                 
        
    #----更新状态---      
    def update(self):
        # 更新位置
        self.pos[0] += self.vel[0]
        self.pos[1] += self.vel[1]
       
        self.rect.x = int(self.pos[0]-self.rect.width//2)  
        self.rect.y = int(self.pos[1]-self.rect.height//2) 
        
    def draw(self, surface):
        if self.image:
            surface.blit(self.image, self.rect)  
        else:
            pygame.draw.rect(surface, WHITE, self.rect)    
        
    def  colliderect(self,sprite):
        return self.rect.colliderect(sprite.rect)
    
    def is_dead(self):
        return self.lives == 0
    
    def set_image(self,image):
        self.image = image
        if  image:
            self.image = pygame.image.load(image)
            image_rect = self.image.get_rect()            
            self.rect = pygame.rect.Rect(self.pos[0]-image_rect.width//2,
self.pos[1]-image_rect.height//2,
                                         image_rect.width,image_rect.height)   

### 5.6.2 Pong游戏

在上述Sprite类的基础上，可以定义Ball类表示乒乓球、Paddle类表示挡板：

In [5]:
class Ball(Sprite):
    def __init__(self, radius=15,pos=(0,0),image=None):
        super().__init__(image, pos) #调用父类的构造函数
        self.init_velocity()
        self.radius = radius
        if not image:
            self.rect = pygame.rect.Rect(pos[0]-radius, pos[1]-radius, 
                2*radius, 2*radius)
               
    def init_velocity(self):
        horizontal = random.randrange(12, 24)/30
        vertical = random.randrange(6, 18)/30
        if random.random()>0.5:   #改变水平速度方向
            horizontal= -horizontal
        if random.random()>0.5:   #改变垂直速度方向
            vertical= -vertical
        self.vel = [horizontal,-vertical]
        
    def draw(self, surface):
        if self.image:
            surface.blit(self.image, self.rect)
        else:
            pygame.draw.circle(surface, WHITE, (int(self.pos[0]),int(self.pos[1])), self.radius, 0)
            
     #----更新状态---        
    def update(self):
        super(Ball,self).update()
        
    
    def backward(self,horizonal = 1):
        if horizonal==1:
            self.vel[0] = - self.vel[0]* 1.1
        elif horizonal==-1:
            self.vel[1] = - self.vel[1]* 1.1
        else :  
            self.vel[0] = -self.vel[0]* 1.1
            self.vel[1] = -self.vel[1]* 1.1             

In [6]:
class Paddle(Sprite):
    def __init__(self,size=(8,80),pos=(0,0),image=None):
        super().__init__(image, pos)       
        self.size = [size[0],size[1]]
        if not image:
            self.rect = pygame.rect.Rect(pos[0]-size[0]//2, pos[1]-size[1]//2,
                                         size[0], size[1])       
            
    def draw(self, surface):
        if self.image:
            surface.blit(self.image, self.rect)
        else:
            GREEN = (0,255,0)
            pygame.draw.rect(surface, GREEN, (self.rect.x, self.rect.y,
                                             self.rect.width, self.rect.height))       


除了2个精灵类型Ball和Paddle，还需要在通用的游戏引擎类GameEngine基础上，派生定义一个表示Pong游戏的的游戏引擎类，假如叫PongGame。

In [7]:
PAD_MOVE_OFFSET = 8
class PongGame(GameEngine):
    def __init__(self, width, height,paddle_width=8,paddle_height=80,ball_radius = 15,title = 'Pong Game',\
                 background = None,foreground = None): 
        super().__init__( width, height,title,background,foreground)
        
        self.scores = [0,0]
        self.ball = Ball(ball_radius,[width//2, height//2])
        
        self.pad_width = paddle_width
        self.pad_height = paddle_height
        self.half_pad_width = paddle_width//2
        self.half_pad_height = paddle_height//2
        
        self.paddle1 = Paddle((self.pad_width,self.pad_height),(self.half_pad_width,height//2))
        self.paddle2 = Paddle((self.pad_width,self.pad_height),(width-self.half_pad_width,height//2))
        self.sprites_list = [self.ball,self.paddle1,self.paddle2]
        
        self.paddle_pressed = 0
   
    def collosion(self): 
        ball = self.sprites_list[0]
        paddle1 = self.sprites_list[1]
        paddle2 = self.sprites_list[2]
        
        width,height = self.surface.get_size()
        
        #上下墙碰撞，水平速度不变，垂直速度相反
        if ball.pos[1] < ball.radius or ball.pos[1] > height - 1 - ball.radius:
            ball.backward(-1)
            
        if ball.pos[0] < (ball.radius + paddle1.rect.width):
            if ball.pos[1] <= paddle1.pos[1] + self.half_pad_height and \
               ball.pos[1] >= paddle1.pos[1] - self.half_pad_height:
                    ball.backward(1)
                    self.scores[0] += 1
            else:
                ball.pos = [self.width//2,self.height//2]
                ball.init_velocity()
                self.scores[1] += 1
        
        if ball.pos[0] >  self.width - 1 - ball.radius - self.pad_width:           
            if ball.pos[1] <= paddle2.pos[1] + self.half_pad_height and  \
               ball.pos[1] >= paddle2.pos[1] - self.half_pad_height:
                    ball.backward(1)
                    self.scores[1] += 1
            else:
                ball.pos = [self.width//2,self.height//2]
                ball.init_velocity()
                self.scores[0] += 1              
    
        # 更新挡板的位置，保持挡板在窗口里
        if paddle1.pos[1] < self.half_pad_height or paddle1.pos[1]+self.half_pad_height> self.height:
            paddle1.pos[1] -=  paddle1.vel[1]
       
        if paddle2.pos[1] < self.half_pad_height or paddle2.pos[1]+self.half_pad_height> self.height:
            paddle2.pos[1] -=  paddle2.vel[1]
        
    # 按下按键的处理函数
    def keydown(self,event):
        if event.key == pygame.K_ESCAPE:  #退出
            return STOP    
        elif event.key == pygame.K_SPACE:  #暂停
            return PAUSE   
        elif event.key == K_UP:
            self.paddle2.vel[1] = -PAD_MOVE_OFFSET
            self.paddle_pressed = 2
        elif event.key == K_DOWN:
            self.paddle2.vel[1] = PAD_MOVE_OFFSET
            self.paddle_pressed = 2           
        elif event.key == K_w:
            self.paddle1.vel[1] = -PAD_MOVE_OFFSET
            self.paddle_pressed =1
        elif event.key == K_s:
            self.paddle1.vel[1] = PAD_MOVE_OFFSET 
            self.paddle_pressed = 1
        return RUNNING

    # 按键弹起的处理函数
    def keyup(self,event):
        if event.key in (K_w, K_s):
           self.paddle1.vel[1] = 0
        elif event.key in (K_UP, K_DOWN):
           self.paddle2.vel[1] = 0          
        return RUNNING

    #绘制得分
    def draw_scores(self, surface):
        self.draw_text(str(self.scores[0]),(10,10))
        self.draw_text(str(self.scores[1]),(self.width-30,10))


最后，可以用一个主程序，测试这个新的游戏PongGame：

In [8]:
#--------主程序--------
WIDTH = 640
HEIGHT = 480  
win = PongGame(WIDTH,HEIGHT)
win.run()


### 5.6.3 仿“雷电战机“游戏

游戏中敌我双方战机、爆炸、声音等资源文件很多，为此，专门定义了一个类GameResource用于保存图像、声音文件的路径。

In [9]:
class GameResource:
    def __init__(self,init_file = None):
        img_dir = 'SpaceInvander/images/'
        sound_dir = 'SpaceInvander/sounds/'
        self.bg_fg = ['map.jpg','fg_map.jpg']
        self.player_images = ['player1.png','player2.png','player1_hit1.png','player1_hit2.png',
                                        'player2_hit1.png','player2_hit2.png',
                              'player_destroy1.png','player_destroy2.png','player_destroy3.png']  
        self.enemy_images = ['Enemy1.png','Enemy2.png','Enemy3.png',
                                      'Enemy1_hit1.png','Enemy1_hit2.png','Enemy2_hit1.png','Enemy2_hit2.png',
                                      'Enemy3_hit1.png','Enemy3_hit2.png','Enemy1_destroy1.png','Enemy1_destroy2.png'
                                      ,'Enemy2_destroy1.png','Enemy2_destroy2.png','Enemy3_destroy1.png','Enemy3_destroy2.png']
        self.boss_images = ['Boss1.png','Boss2.png','Boss1_hit1.png','Boss1_hit2.png',
                                     'Boss2_hit1.png','Boss2_hit2.png','Boss_destroy1.png','Boss_destroy2.png',
                                     'Boss_destroy3.png','Boss_destroy4.png','Boss_destroy5.png','Boss_destroy6.png',
                                     'Boss_destroy7.png','Boss_destroy8.png','Boss_destroy9.png','Boss_destroy10.png']
        
        self.bullet_images = ['bullet_blue.png','bullet_purple.png','bullet_yellow.png']
        self.enemy_bullet_images = ['EnemyBullet1.png','EnemyBullet2.png','EnemyBullet3.png']    
        
        
        self.bg_fg = [img_dir+x for x in self.bg_fg]
        self.player_images = [img_dir+'player_images/'+x for x in self.player_images]
        self.enemy_images = [img_dir+'enemy_images/'+x for x in self.enemy_images]
        self.boss_images = [img_dir+'enemy_images/'+x for x in self.boss_images]
        self.bullet_images = [img_dir+'bullet_images/'+x for x in self.bullet_images]
        self.enemy_bullet_images = [img_dir+'bullet_images/'+x for x in self.enemy_bullet_images]
                      
        self.sounds = {'bg_music':'bg_music.mp3','shot_sound':'shot.wav',
                       'player_hited_sound':'user_down.wav',
                       'crash_sound':'explosion.wav',
                       'explosion_sound':'explosion.wav','user_down_wav':'user_down.wav' 
                      }
        
        self.sounds.update((k, sound_dir+v ) for k,v in self.sounds.items())                       

游戏引擎类SpaceInvander类从GameResource类对象接受这些资源文件的路径。初始化相应的数据。creat_sprites()将来可以用于创建游戏中的精灵，draw_background()绘制游戏的背景画面，为了让背景不断移动，在原始背景图片的上方再拼接一个同样的图片，当原始背景图片向下移动时，上方的图片也一起向下移动，一旦原始图片和复制图片整个移出或移入，再让它们回到起始位置，重复这种移动，造成一个无限大的背景的错觉。初始化读取了背景音乐文件后就开始播放背景音乐。

In [11]:
from pygame.time import get_ticks
from pygame import mixer
    
class SpaceInvander(GameEngine):
    def __init__(self, width, height,title = 'SpaceInvander',resource = GameResource()): 
        super().__init__( width, height,title)    
        self.resource =resource    
        self.background = pygame.image.load(self.resource.bg_fg[0]).convert() 
        self.background2 = self.background.copy()
        self.pos_y1  = -1024
        self.pos_y2  = 0
        self.bg_roll_speed_factor = 0.1
        
        #播放背景音乐
        mixer.init()   #sound
        pygame.mixer.music.load(self.resource.sounds['bg_music'])  
        pygame.mixer.music.play(1) 

        self.player_id = 0
        self.player = None
        self.enemy_id = 0
        self.player = None
        self.boss_id = 0
        self.boss = None
        self.aliens = []
        self.bullets = []
        self.bullets_boss = []        
        self.explosions = []
        self.score = 0
        self.creat_sprites()       
    
    def creat_sprites(self):
        pass
   
    def draw_background(self, surface):
        surface.fill(self.bg_color)   # clear surface to green
        surface.blit(self.background, (0,0))
        surface.blit(self.background, (0, self.pos_y1))
        surface.blit(self.background2, (0, self.pos_y2))
    
    def update(self):
        super().update()
        self.pos_y1 += self.bg_roll_speed_factor
        self.pos_y2 += self.bg_roll_speed_factor

        if self.pos_y1 > 0:
            self.pos_y1 = -1024
        if self.pos_y2 > 1024:
            self.pos_y2 = 0 
            
    def game_intro(self): 
        logo_img = self.background      
        intro = True
        while intro:
            #检查是否退出
            for event in pygame.event.get():
                if event.type == pygame.QUIT or event.type == pygame.KEYDOWN and event.key==pygame.K_ESCAPE: 
                    intro = False 
                    break            
            
            self.surface.blit(logo_img, (0, 0))
            text = "Press Esc or window X to Enter Game..."
            self.draw_text(text,(12,self.height//2)) 
            pygame.display.flip()         # pygame.display.update()
        pygame.mixer.music.stop()    
        
    def draw_scores(self, surface):     
        self.draw_text(str(self.score),(self.width-30,10))
   
WIDTH = 400
HEIGHT = 600  
resource = GameResource()
game = SpaceInvander(WIDTH,HEIGHT,'SpaceInvander',resource)
game.run()

运行上述程序，出现图5-9 a)的开始画面，并播放背景音乐。按下Esc键或鼠标点击窗口x，停止音乐播放，进入背景变化的游戏主画面（图5-9 b））。

![](imgs/5_9.png)

**注**：本书资源请在电子工业出版社官网下载

游戏的精灵主要有：玩家（我方）战机Player、敌方母机Boss、敌方战机、敌方子弹、我方子弹。除了具有精灵已有的属性如位置pos、速度vel、更新update()、绘制draw()外，我方战机、敌方战机都具有一些共同属性：生命值lives、射击shot()、死亡判断函数is_dead()。因此，先定义一个从Sprite派生的表示战机的精灵类Fighter。

In [12]:
class Fighter(Sprite):
    def __init__(self,image=None, hit_images=None,destroy_images=None,                 
                 bullet_images=None,shot_sound = None,crash_sound = None,
                 bullet_speed = 1,move_speed = 0.5,lives=3,power = 3,pos=(0,0)):   
        super().__init__(image, pos) 
        self.hit_images = hit_images          #射中图像
        self.destroy_images = destroy_images  #销毁图像
        self.lives = lives                     #生命值
        self.power = 1   #战斗力
        self.move_speed  = move_speed  #运动速度系数
        
        self.bullet_images = bullet_images
        self.bullet_id = 0        #战斗力增强后可以升级子弹
        self.bullet = None
        self.bullet_speed = bullet_speed
        self.shot_sound = shot_sound
        
        self.crash_sound = crash_sound 
        self.hit_id=0       
        
    def shot(self):
        self.bullet = Bullet(self.bullet_images[self.bullet_id],
                    (self.rect.centerx, self.rect.top),(0,self.bullet_speed) )  
        if self.shot_sound: 
            self.shot_sound.play()
        return self.bullet 
    
    def is_dead(self):
        return self.lives==0
    
    def hitted(self):         #被击中时的处理
        self.lives -= 1   
        if self.crash_sound:   
            self.crash_sound.play()
        if self.hit_id<len(self.hit_images):
            self.set_image(self.hit_images[self.hit_id]) 
        self.hit_id +=1


同样，为敌我双反的子弹可以定义一个共同的子弹类Bullet(假设敌我双方的子弹类型是一样的)。

In [13]:
class Bullet(Sprite):
    def __init__(self,image=None, pos=(0,0),velocity = (0,0)):
        super().__init__(image, pos) 
        self.pos[1] -= self.rect.height//2
        self.rect.y -= self.rect.height//2
        self.vel = [velocity[0],velocity[1]]        
       
    def update(self):
        super().update()      
        w, h = pygame.display.get_surface().get_size()
        if self.rect.y < 1 or self.rect.y>= h:           
             self.lives = 0 


玩家Player除具有Fighter的共同属性外，还具有如下功能：加油powUp()。同时修改 update()位置更新方法，防止Player跑出窗口区域。

In [14]:
class Player(Fighter):                  
    def powerUp(self):
        self.power += 1
        self.power_time = pygame.time.get_ticks()

    def update(self):
        # 更新位置
        w, h = pygame.display.get_surface().get_size()
        x = self.pos[0] + self.vel[0]
        y = self.pos[1] + self.vel[1]
        if x>self.rect.width//2 and x< w- self.rect.width//2:
            self.pos[0] = x
        if y>self.rect.height//2 and y< h- 20+1:
            self.pos[1] = y
        self.rect.x = int(self.pos[0]-self.rect.width//2) 
        self.rect.y = int(self.pos[1]-self.rect.height//2)


现在修改SpaceInvander，在creat_sprites()方法里添加一个Player，然后定义事件处理函数，当按下空格键时就发射子弹、移动键时就移动。

In [15]:
class SpaceInvander2(SpaceInvander):
    #...
    def creat_player(self):            
            image = self.resource.player_images[self.player_id]
            self.num =2
            hit_id =  self.num+2*self.player_id # 2*(self.player_id+1)
            hit_images = [self.resource.player_images[hit_id],self.resource.player_images[hit_id+1]]
            des_id = self.num+2*self.num+2*self.player_id
            des_images = [self.resource.player_images[des_id],self.resource.player_images[des_id+1]]
            
            shot_sound = mixer.Sound(self.resource.sounds['shot_sound'])            
            crash_sound = mixer.Sound(self.resource.sounds['crash_sound'])            
            
            shoot_delay = 250
            bullet_speed = -1
            move_speed= 0.5
            lives=3 
            power = 3
            pos= (self.width//2, self.height-20)
                
            self.player = Player(image,hit_images, des_images,self.resource.bullet_images,
                                 shot_sound,crash_sound,bullet_speed, 
move_speed,lives,power,pos)                                      
    def creat_sprites(self):  
        self.creat_player()
        self.add_sprite(self.player)          
            
     # keydown handler
    def keydown(self,event):
        if event.key == pygame.K_ESCAPE:  #退出
            return 0    
        elif event.key == pygame.K_SPACE:  
            bullet = self.player.shot()
            if bullet:
                self.bullets.append(bullet)
                self.add_sprite(bullet)
                self.player.bullet = None
                
        elif event.key == K_RIGHT:
            self.player.vel[0] = self.player.move_speed
            #self.player.update()           
        elif event.key == K_LEFT:
            self.player.vel[0] = -self.player.move_speed
        elif event.key == K_DOWN:
            self.player.vel[1] = self.player.move_speed            
        elif event.key == K_UP:
            self.player.vel[1] = -self.player.move_speed
        elif event.key == K_w:
            pass
        elif event.key == K_s:
            pass 
        return 1

    #keyup handler
    def keyup(self,event):
        if event.key in (K_w, K_s):
            pass
        elif event.key in (K_RIGHT, K_LEFT):
            self.player.vel[0] = 0           
        elif event.key in (K_DOWN,K_UP):
            self.player.vel[1] = 0
        return 1
    
    def update(self):
        super().update()
              
        if self.player and self.player.is_dead():               
                self.player = None 
                    
        for bullet in self.bullets:
            if bullet.is_dead():               
                self.bullets.remove(bullet) 
              
game = SpaceInvander2(WIDTH,HEIGHT)
game.run()

运行这个程序，将显示如图5-10的游戏画面，我方的战机可以发射子弹，但还没有敌机，得分也为0。
![](imgs/5_10.png)

假设敌机有2种：普通的敌机和大Boss敌机。开始时，随机出现的是一些敌机，每个敌机会发射子弹，当我方针战机击溃一定数目的敌机后，敌方的大Boss敌机就会出现，大Boss同样会发射子弹，并缓慢地向我逼近。当然，大Boss敌机发射的子弹类型可能和我方的子弹类型不一定，但本书为了简单起见，就重复用同一个Bullet类表示双方的子弹，只是给它们不同的形状。
因此，从战机类Fighter派生出Enemy类和Boss类分别表示敌方普通战机和大Boss战机，敌机移动和发射子弹不是由用户控制的，而是其自主的行为，因此，在update()里让敌机随机移动和发射子弹：让敌机随机移动，经过一定时间开始射击。


In [18]:
class Enemy(Fighter):
    def __init__(self,image=None, hit_images=None,destroy_images=None,                 
                 bullet_images=None,shot_sound = None,crash_sound = None,
                 shoot_delay= 250,shoot_interval = 2500,bullet_speed = 1,move_speed = 0.5,
                 lives=3,power = 3,pos=(0,0)):
        super().__init__(image,hit_images,destroy_images,bullet_images,shot_sound,crash_sound,
                         bullet_speed ,move_speed,lives,power,pos)
        self.shoting = False                   # 是否射击标志
        self.shoot_delay = shoot_delay         # 连续射击延迟    
        self.shoot_interval = shoot_interval     # 2次射击间隔
        self.shot_last = pygame.time.get_ticks()
        self.shot_start = pygame.time.get_ticks()
      
    def update(self):
        self.shot()
        self.generate_velocity()         
        super().update() 
        
    def generate_velocity(self):        
        move = random.random()>0.8
        acc = random.random()>0.5    
        if(move):
            self.vel[1] = random.random()*self.move_speed
            self.vel[0] = (random.random()-0.5)*self.move_speed
            if acc: self.vel[0] *=5
    
    def shot(self):
        now = pygame.time.get_ticks()
        if now - self.shot_start > self.shoot_interval: #超过一定时间，改变射击标志
            self.shoting = not self.shoting
            self.shot_start = now
        if not self.shoting: return None
        if now - self.shot_last > self.shoot_delay:   #开始射击，修改最后一次设计时间
            super().shot()
            self.shot_last = now  
    
class Boss(Enemy):
    pass     

现在，需要修改SpaceInvander的creat_sprites()和update()函数。

In [19]:

import copy

def copy_enemy(enemy,enemy2):    
        enemy2.vel = copy.copy(enemy.vel)   
        enemy2.pos = copy.copy(enemy.pos)  
        enemy2.rect = copy.copy(enemy.rect)  
        
    
class SpaceInvander3(SpaceInvander2):   
    def creat_sprites(self):       
        super().creat_sprites()
        self.boss = None
        self.enemys = []
        self.bullets_enemy = []
        self.enemy_last_time =  pygame.time.get_ticks()
        self.enemy_time_interval = 5000
       
    def creat_enemies(self):
        if self.running != RUNNING: return
        self.enemy_id = 0
        self.num = 3
        image = self.resource.enemy_images[self.enemy_id]
        hit_id = self.num+2*self.enemy_id
        hit_images = [self.resource.enemy_images[hit_id],self.resource.enemy_images[hit_id+1]]
        des_id = self.num+2*self.num+ 2*self.enemy_id
        des_images = [self.resource.enemy_images[des_id],self.resource.enemy_images[des_id+1]]
        shot_sound = None #mixer.Sound(self.resource.sounds['shot_sound'])     
        crash_sound = mixer.Sound(self.resource.sounds['crash_sound'])   
        shoot_delay= 500
        shoot_interval = 3000
        
        bullet_speed = 0.5
        move_speed= 0.2
        lives=1 
        power = 1
        pos= (self.width/2, 20)      
            
        enemy = Enemy(image,hit_images, des_images,self.resource.enemy_bullet_images,
                                 shot_sound,crash_sound,shoot_delay,shoot_interval,bullet_speed,
                                 move_speed,lives,power,pos)        
        enemy.image = pygame.transform.rotate(enemy.image, 180)
        enemy.last_shot_time = pygame.time.get_ticks() 
        enemy.shot_delay = 1000
    
        enemy.pos[0] = self.width/2+ (random.random()-0.5)*0.2*self.width #x随机偏移
        enemy.pos[1] = random.random()*self.num *enemy.rect.height      #y随机偏移   
       # self.add_sprite(enemy)
       # return enemy
        
        enemy2 = copy.copy(enemy) 
        enemy3 = copy.copy(enemy) 
        copy_enemy(enemy,enemy2)
        copy_enemy(enemy,enemy3)
        
        enemy2.pos[0] = random.uniform(enemy.rect.width*2,enemy.pos[0]-enemy.rect.width)
        enemy2.pos[1] += (random.random()-0.5)*enemy.rect.height*0.5
        enemy3.pos[0] = random.uniform( enemy.pos[0]+enemy.rect.width,self.width-enemy.rect.width*2)
        enemy3.pos[1] += (random.random()-0.5)*enemy.rect.height*0.5
           
        enemys = [enemy,enemy2,enemy3]
        for e in enemys:
            e.update()
            self.add_sprite(e)
        self.enemys += enemys

       
    
    def creat_Boss(self):
        if self.running != RUNNING: return
        self.boss_id = 0
        image = self.resource.boss_images[self.boss_id]
        hit_id = self.num+2*self.boss_id
        hit_images = [self.resource.boss_images[hit_id],self.resource.boss_images[hit_id+1]]
        des_id = self.num+2*self.num+ 2*self.boss_id
        des_images = [self.resource.boss_images[des_id],self.resource.boss_images[des_id+1]]
        shot_sound = None #mixer.Sound(self.resource.sounds['shot_sound'])     
        crash_sound = mixer.Sound(self.resource.sounds['crash_sound'])   
        shoot_delay= 250
        shoot_interval = 2500
        bullet_speed = 1
        move_speed= 0.1
        lives=1 
        power = 1
        pos= (self.width/2, -30) 
        self.boss = Boss(image,hit_images, des_images,self.resource.enemy_bullet_images,
                                 shot_sound,crash_sound,shoot_delay,shoot_interval,bullet_speed,
                                 move_speed,lives,power,pos)
     
        self.add_sprite(self.boss)
    
    def update(self):  
      
        for enemy in self.enemys:
            if enemy.pos[1]>self.height:
                enemy.lives=0
                continue
            if enemy.bullet:
                self.add_sprite(enemy.bullet)                   
                self.bullets_enemy.append(enemy.bullet)
                enemy.bullet = None  
        
        if self.boss:
            if self.boss.pos[1]>self.height:
                self.remove_sprite(self.boss)
                self.boss = None
            elif self.boss.bullet:
                self.add_sprite(self.boss.bullet) 
                self.bullets_enemy.append(self.boss.bullet)
                self.boss.bullet = None  
        elif(self.score>5):
            self.creat_Boss()   #成绩大Boss敌机
    
        for enemy in self.enemys:
            if enemy.is_dead():
                self.enemys.remove(enemy) 
                #self.remove_sprite(enemy)
        
        for bullet in self.bullets_enemy:
            if bullet.is_dead():
                self.bullets_enemy.remove(bullet) 
                #self.remove_sprite(bullet)
                
        for bullet in self.bullets:
            if bullet.is_dead():
                self.bullets.remove(bullet) 
                #self.remove_sprite(bullet)
            
        #新建新的敌机
        now =  pygame.time.get_ticks()       
        if now - self.enemy_last_time> self.enemy_time_interval:
            self.creat_enemies()   #经过一段时间，创建一批敌机
            self.enemy_last_time = now
       
        super().update()
      
game = SpaceInvander3(WIDTH,HEIGHT)
game.run()


运行上述程序，将出现如图5-11的画面： 
![](imgs/5_11.png)

但当子弹射向敌机或敌机射向我方战机或发送碰撞时，都没有任何处理。下面增加碰撞处理，当子弹击中目标时，产生一个爆炸效果，并销毁被击中目标。首先需要定义一个爆炸的精灵，爆炸精灵Explosion模拟爆炸的动画效果，因此需要多福图像，当然可以用Pygame的精灵类轻松完成这个爆炸动画效果。这里故意完全从头编写如何用多个图片模拟动画效果，以便读者将来能更好地从底层理解实现原理。Explosion构造函数接受一个多个图片路径的list对象。

In [20]:
class Explosion(Sprite):
    def __init__(self, images, position,explosion_wav):
        self.images = images        
        super(Explosion, self).__init__(images[0], position)
        self.frame = 0
        self.last_update = pygame.time.get_ticks()
        self.frame_rate = 50  
        self.lives = len(self.images)
        self.sound_explosion = mixer.Sound(explosion_wav)
        
    def update(self):
        now = pygame.time.get_ticks()
        if now - self.last_update > self.frame_rate: #经过一定时间，更新爆炸画面
            self.last_update = now
            self.frame += 1
            if self.frame == len(self.images): #爆炸动画图片播放完，炸弹就消亡
                self.lives = 0
            else:              
                self.set_image( self.images[self.frame]) 
                super().update()


下面添加碰撞检测功能，但发生碰撞时，生命值减少1，当生命值为0时，就销毁产生爆炸效果。

In [ ]:
运行上述程序，如图5-12所示，现在就可以检测战机和子弹的碰撞了，当一个战机的生命值为0时就会发生爆炸而销毁。限于篇幅，对于战机被击中，战机损坏的图像等读者可自行添加，还有战机、子弹等超出窗口怎么处理？这些都留待读者自己去解决，作者将在博客网上放上更完善的雷电游戏。
![](imgs/)